In [74]:
import os
import re
from pandas.api.types import CategoricalDtype
import numpy as np
import pandas as pd
import math
import csv
from unidecode import unidecode

In [189]:
proceeding_ID_Paper = "12780"
proceeding_ID_EA = "12781"

In [229]:
VENUE = "tp23"
setup = [["Paper", "paper", "tp23a", "Associated Chairs","Reviewers"],
         ["Late-Breaking Work", "lbw", "tp23k", "Associated Chairs", "Reviewers"],
         ["Demonstrations", "demo", "tp23o", "Jury Member", "Reviewers"],
         ["Doctoral Consortium", "dc", "tp23j", "Jury Member", "Reviewers"],
         ["Workshop", "ws", "tp23d", "Jury Member", "Reviewers"],
        ]

dfVenues = pd.DataFrame(setup, columns=["Name", "Code", "File", "NameCommittee", "NameReviewers"])
dfVenues

,Name,Code,File,NameCommittee,NameReviewers
0,Paper,paper,tp23a,Associated Chairs,Reviewers
1,Late-Breaking Work,lbw,tp23k,Associated Chairs,Reviewers
2,Demonstrations,demo,tp23o,Jury Member,Reviewers
3,Doctoral Consortium,dc,tp23j,Jury Member,Reviewers
4,Workshop,ws,tp23d,Jury Member,Reviewers


In [230]:
f = "tp23k"

vanues = {"e":"src", "f":"sdc", "g":"sgc", "h":"lbw", "i":"ws", "j":"dc", "k":"dein", "l":"panel", "m":"cs", "n":"crs", "o":"altChi", "q":"sig"}


In [231]:
def getCommittee(venueCode):
    path = f"./data-PCS/{venueCode}_committee.csv"
    if (not os.path.isfile(path)):
        print(f"{venueCode} has no committee file")
        return []
    
    df = pd.read_csv(path)
    df = df[df["Reviews assigned"] != 0]
    
    if (len(df) == 0):
        print(f"{venueCode} has no reviewers")
        return []

    lstText = []
    df["Family name"] = df["Family name"].str.title()
    df["First name"] = df["First name"].str.title()
    #df["Middle name"] = df["Middle name"].str.title()
    df = df.sort_values(["Family name", "First name", "Middle initial"])

    for i, e in df.iterrows():
        for i in range (1,7):
            if isinstance(e[f"Affil {i} Institution"], str):
                break
        aff = ""
        if isinstance(e[f"Affil {i} Institution"], str):
            aff = f'{e[f"Affil {i} Institution"]}, {e[f"Affil {i} Country"]}'

        aff = aff.replace("&", "\\&")
        if isinstance(e["Middle initial"], str):
            lstText.append(f'{e["First name"]} {e["Middle initial"]} {e["Family name"]}, \\emph{{{aff}}}\\\\')
        else:
            lstText.append(f'{e["First name"]} {e["Family name"]}, \\emph{{{aff}}}\\\\')

    if (len(df) > 50):   
        lstText.insert(0, "\\begin{multicols}{2}")
        lstText.append("\end{multicols}")
    else:
        
        lstText.insert(0, "%\\begin{multicols}{2}")
        lstText.append("%\end{multicols}")
    lstText.append("")
    return lstText

In [232]:
def getReviews(venueCode):
    path = f"./data-PCS/{venueCode}_reviewers.csv"
    if (not os.path.isfile(path)):
        print(f"{venueCode} has no reviwer file")
        return []
    
    df = pd.read_csv(path)
    df = df[df["Reviews assigned"] != 0]
    
    if (len(df) == 0):
        print(f"{venueCode} has no reviewers")
        return []
    
    # Ensure that all names start with a capital first latter.
    df["Family name"] = df["Family name"].apply(lambda x: x[0].title()+x[1:] if len(x)>2 else x)
    df["First name"] = df["First name"].apply(lambda x: x[0].title()+x[1:] if len(x)>2 else x)
    if "Middle name"in df.columns:
        df["Middle name"] = df["Middle name"].apply(lambda x: x[0].title()+x[1:] if len(x)>2 else x)
    
    df = df.sort_values(["Family name", "First name", "Middle initial"])
    lstText = []
    lstText.append("\\begin{multicols}{3}")
    for i, e in df.iterrows():
        if isinstance(e["Middle initial"], str):
            lstText.append(f'{e["First name"]} {e["Middle initial"]} {e["Family name"]}\\\\')
        else:
            lstText.append(f'{e["First name"]} {e["Family name"]}\\\\')
    lstText.append("\\end{multicols}")
    lstText.append("")
    return lstText

In [233]:
for i, e in dfVenues.iterrows():
    lstExport = []
    lstExport.append("% If this venue/track has subcommittees, you might want to split the \subsection{Committee Member} into different \subsubsections for the different committees.")
    lstExport.append("")    
    lstExport.append(f"\\subsection{{{e.Name} Chairs}}")
    lstExport.append("First Chair Full Name, \emph{Affiliation, Country}\\\\")
    lstExport.append("Second Chair Full Name, \emph{Affiliation, Country}")
    lstExport.append("")
    lstExport.append(f"%\subsection*{{Assistants to {e.Name} Chairs}}")
    lstExport.append("%First Assistants Full Name, \emph{Affiliation}\\\\")
    lstExport.append("%Second Assistants Full Name, \emph{Affiliation}")
    lstExport.append("")
    lstExport.append("")
    
    commitee = getCommittee(e.File)
    if (len(commitee) > 0):
        lstExport.append(f"\subsection{{{e.NameCommittee}}}")
        lstExport.extend(commitee)

    reviewers = getReviews(e.File)
    if (len(reviewers) > 0):
        lstExport.append(f"\subsection{{{e.NameReviewers}}}")
        lstExport.extend(reviewers)
    
    if (len(lstText) > 0):
        with open(f'committee/committee-{e.Code}.tex', 'w') as fp:
            fp.write('\n'.join(lstExport))
            

tp23o has no reviewers
tp23j has no reviewers
tp23d has no reviewers


In [234]:
dfTAPS = pd.read_csv("export_12781.csv")

file1 = open(f"./{proceeding_ID_EA}.csv", 'r')
lines = file1.readlines()
ids = []
lstLines = []
for x in lines[1:]:
    e = x[1:].split('","')

    if (not("WITHDRAWN" in x)):
        lstLines.append(e)
        ids.append(x.split(",")[0].replace('"', ""))
    else:
        print("WITHDRAWN", x.split(",")[0].replace('"', ""))
    

dfACM = pd.DataFrame(lstLines)
dfACM.columns = ["ID","Title","Author","EmailX","DL Paper Type","Rights Granted","Third Party","Aux. Material","Video Recording","Artistic Images","Govt. Employees","Open Access","DOI","Authorizer","Statement","CC License","Non-ACM Copyright"]
dfACM['Email2'] = dfACM.EmailX.apply(lambda x: x.split(" ")[0])
dfACM['Signed'] = dfACM['Non-ACM Copyright'].apply(lambda x: x[:-3]) != "load For"
dfACM["TitleOld"] = dfACM.Title
dfACM.Title = dfACM.TitleOld.apply(lambda x: x.split(" \\setcopyright{")[0])
dfACM["VenueShort"] = dfACM.ID.apply(lambda x: re.sub(r'[0-9]', '', x))

dfACM["TitleRaw"] = dfACM.Title.str.replace('"', '')
dfACM["TitleRaw"] = dfACM.TitleRaw.str.replace('[', '', regex=False)
dfACM["TitleRaw"] = dfACM.TitleRaw.str.lower()
print(dfACM["TitleRaw"].apply(lambda x: x[0]).unique())

dictVenue = {'pn':"Paper",
    'ws':"Workshops / Symposia",
    'cs':'Case Studies of HCI in Practice',
    'crs':'Courses',
    'dc':'Doctoral Consortium',
    'alt': 'alt.chi',
    'sig': 'Special Interest Groups',
    'panel': 'Panels',
    'sgc': 'Student Game Competition',
    'sdc': 'Student Design Competition',
    'dein': 'Interactivity',
    'int': 'Interactivity',
    'src': 'Student Research Competition',
    'lbw': 'Late-Breaking Work'}

dfACM.VenueShort = dfACM.VenueShort.str.replace("dein", "int")

dfACM["VenueName"] = dfACM.VenueShort.map(dictVenue)

myOrder = CategoricalDtype(
    ['pn', 'lbw', 'ws', 'cs', 'alt', 'dein', 'int', 'dc', 'sig', 'panel', 'crs',  'src', 'sdc', 'sgc'], 
    ordered=True
)

dfACM = dfACM.drop_duplicates("ID")

df = dfACM

print(len(df))
df = pd.merge(df, dfTAPS[["ID", "PaperID"]], left_on="ID", right_on="ID")
print(len(df))

df.VenueShort = df.VenueShort.astype(myOrder)
df = df.sort_values(["VenueShort", "TitleRaw"])

text = []
lastVenueShort = ""
counter = 1
counterTOC = 1

tapsTOC=[]
lstAuthorIndex = []
for i, e in df.iterrows():
    if (e.VenueShort != lastVenueShort):
        
        if counter != 1:
            text.append(f'\\end{{enumerate}}')
            text.append("")
            
        text.append(f'\\subsection{{{e.VenueName}}}')
        text.append("")
        text.append(f'\\begin{{enumerate}}')
        
        
        lastVenueShort = e.VenueShort
        counter = 1
    
    
    TOC_ID = f"{e.VenueShort.upper()}{counter:03}"
    text.append(f'\\item[\\href{{{e.DOI}}}{{\\textbf{{{TOC_ID}}}}}]')
    
    xx =f'\\href{{{e.DOI}}}{{\\textbf{{{e.Title}}}}}\\\\'
    text.append(xx.replace("&", "\\&").replace("#", "\\#"))
    for x in e.Author.split(";"):
        x = x.split(":")
        xx = f"{x[0]} \\emph{{({x[1]})}} \\\\"
        lstAuthorIndex.append([x[0], TOC_ID])
        text.append(xx.replace("&", "\\&").replace("#", "\\#").replace(",", ", ").replace("  ", " "))
    
    text.append("")
    
    
    tapsTOC.append([counterTOC, e.PaperID, e.ID, TOC_ID, e.Title, e.VenueName])
    counter = counter + 1
    counterTOC = counterTOC + 1

text.append(f'\\end{{enumerate}}')

if (len(text) > 0):
    with open(f'./content/content-ea.tex', 'w') as fp:
        fp.write('\n'.join(text))
        
        

with open('./TOC-CHIEA23.csv', 'w', newline='') as csvfile:
    w = csv.writer(csvfile, delimiter=',',
                            quotechar='"')
    w.writerow(["ID","PaperID","PCS_ID","TOC_ID","Title","Session"])
    for e in tapsTOC:
        w.writerow(e)
        
        
dfAI = pd.DataFrame(lstAuthorIndex)
dfAI.columns = ["Name", "Submission"]
dfAI = dfAI.groupby("Name").Submission.apply(lambda x: ", ".join(x))
dfAI = dfAI.reset_index()
dfAI["NameRaw"] = dfAI.Name.str.replace('"', '')
dfAI["NameRaw"] = dfAI.NameRaw.apply(lambda x: unidecode(x))
dfAI["NameRaw"] = dfAI.NameRaw.str.replace('[', '', regex=False)
dfAI["NameRaw"] = dfAI.NameRaw.str.lower()
dfAI = dfAI.sort_values("NameRaw")

if (len(dfAI) > 0):
    with open(f'./content/index-ea.tex', 'w') as fp:
        fp.write('\\begin{multicols}{2}\n')
        for i, e in dfAI.iterrows():
            fp.write(f'{e.Name} \dotfill {e.Submission}\\\\\n')
            
        fp.write('\\end{multicols}')

FileNotFoundError: [Errno 2] No such file or directory: 'export_12781.csv'

In [199]:
dfAI.NameRaw

0              aale luusua
1       aarnout brombacher
2          aaron j quigley
3               aaron lyon
4           aaron schecter
               ...        
2354      zinaida benenson
2355               ziru li
2356          ziyuan jiang
2357     zjenja doubrovski
2358        zoya bylinskii
Name: NameRaw, Length: 2360, dtype: object

In [156]:
lastVenueShort = ""
for i, e in df.iterrows():
    if (e.VenueShort != lastVenueShort):
        print(e.VenueName)
        lastVenueShort = e.VenueShort

Late-Breaking Work
Workshops / Symposia
Case Studies of HCI in Practice
alt.chi
Interactivity
Doctoral Consortium
Special Interest Groups
Panels
Courses
Student Research Competition
Student Design Competition
Student Game Competition


In [200]:
dfTAPS = pd.read_csv("export_12780.csv")



import json

with open("qoala-chi23.json", 'r') as f:
    qoala = json.load(f)
    
dfQPapers = pd.DataFrame(qoala["contents"])
dfQPapers = dfQPapers[dfQPapers.importedId.apply(lambda x: x.startswith("chi23b"))]
print(f'Error in {len(dfQPapers[dfQPapers.sessionIds.apply(lambda x: len(x) > 1)])} papers')
dfQPapers.sessionIds = dfQPapers.sessionIds.apply(lambda x: x[0])

dfQSessions = pd.DataFrame(qoala["sessions"])
dfQSessions = dfQSessions.rename(columns={"id":"sessionId", "name":"SessionName"})

dfQ = pd.merge(dfQPapers, dfQSessions[["sessionId", "SessionName"]], left_on="sessionIds", right_on="sessionId", how="left")[["importedId", "title", "SessionName"]]

dfQ["ID"] = dfQ.importedId.apply(lambda x: x.replace("chi23b-", "pn"))
dfQ


file1 = open(f"./{proceeding_ID_Paper}.csv", 'r')
lines = file1.readlines()
ids = []
lstLines = []
for x in lines[1:]:
    e = x[1:].split('","')

    if (not("WITHDRAWN" in x)):
        lstLines.append(e)
        ids.append(x.split(",")[0].replace('"', ""))
    else:
        print("WITHDRAWN", x.split(",")[0].replace('"', ""))
    

dfACM = pd.DataFrame(lstLines)
dfACM.columns = ["ID","Title","Author","EmailX","DL Paper Type","Rights Granted","Third Party","Aux. Material","Video Recording","Artistic Images","Govt. Employees","Open Access","DOI","Authorizer","Statement","CC License","Non-ACM Copyright"]
dfACM['Email2'] = dfACM.EmailX.apply(lambda x: x.split(" ")[0])
dfACM['Signed'] = dfACM['Non-ACM Copyright'].apply(lambda x: x[:-3]) != "load For"
dfACM["TitleOld"] = dfACM.Title
dfACM.Title = dfACM.TitleOld.apply(lambda x: x.split(" \\setcopyright{")[0])
dfACM["VenueShort"] = dfACM.ID.apply(lambda x: re.sub(r'[0-9]', '', x))

dfACM["TitleRaw"] = dfACM.Title.str.replace('"', '')
dfACM["TitleRaw"] = dfACM.TitleRaw.str.replace('[', '', regex=False)
dfACM["TitleRaw"] = dfACM.TitleRaw.str.replace('`', '', regex=False)
dfACM["TitleRaw"] = dfACM.TitleRaw.str.replace('“', '', regex=False)
dfACM["TitleRaw"] = dfACM.TitleRaw.str.replace("'", '', regex=False)
dfACM["TitleRaw"] = dfACM.TitleRaw.str.replace('(', '', regex=False)
dfACM["TitleRaw"] = dfACM.TitleRaw.str.replace('#', '', regex=False)
dfACM["TitleRaw"] = dfACM.TitleRaw.str.lower()
print(dfACM["TitleRaw"].apply(lambda x: x[0]).unique())

dictVenue = {'pn':"Paper",
    'ws':"Workshops / Symposia",
    'cs':'Case Studies of HCI in Practice',
    'crs':'Courses',
    'dc':'Doctoral Consortium',
    'alt': 'alt.chi',
    'sig': 'Special Interest Groups',
    'panel': 'Panels',
    'sgc': 'Student Game Competition',
    'sdc': 'Student Design Competition',
    'dein': 'Interactivity',
    'int': 'Interactivity',
    'src': 'Student Research Competition',
    'lbw': 'Late-Breaking Work'}

dfACM.VenueShort = dfACM.VenueShort.str.replace("dein", "int")

dfACM["VenueName"] = dfACM.VenueShort.map(dictVenue)

myOrder = CategoricalDtype(
    ['pn', 'lbw', 'ws', 'cs', 'alt', 'dein', 'int', 'dc', 'sig', 'panel', 'crs',  'src', 'sdc', 'sgc'], 
    ordered=True
)
dfACM.VenueShort = dfACM.VenueShort.astype(myOrder)



dfACM = dfACM.drop_duplicates("ID")
df = dfACM

print(len(df))
df = pd.merge(df, dfQ, on="ID")

print(len(df))
df = pd.merge(df, dfTAPS[["ID", "PaperID"]], left_on="ID", right_on="ID")
print(len(df))

df = df.sort_values(["SessionName", "TitleRaw"])

text = []
lastVenueShort = ""
counter = 1

tapsTOC = []

lastSessionName = ""

lstAuthorIndex = []
for i, e in df.iterrows():

        
    if (e.SessionName != lastSessionName):
        
        if counter != 1:
            text.append(f'\\end{{enumerate}}')
            text.append("")
            
        temp = e.SessionName.replace("&", "\\&").replace("#", "\\#").replace("_", " ")
        
        text.append(f'\\subsection{{{temp}}}')
        text.append("")
        text.append(f'\\begin{{enumerate}}')
        
        
        lastSessionName = e.SessionName
    
    TOC_ID = f"Paper{counter:03}"
    text.append(f'\\item[\\href{{{e.DOI}}}{{\\textbf{{{TOC_ID}}}}}]')
    
    xx =f'\\href{{{e.DOI}}}{{\\textbf{{{e.Title}}}}}\\\\'
    text.append(xx.replace("&", "\\&").replace("#", "\\#"))
    for x in e.Author.split(";"):
        x = x.split(":")
        xx = f"{x[0]} \\emph{{({x[1]})}} \\\\"
        lstAuthorIndex.append([x[0], TOC_ID])
        text.append(xx.replace("&", "\\&").replace("#", "\\#").replace(",", ", ").replace("  ", " "))
    
    text.append("")
    
    tapsTOC.append([counter, e.PaperID, e.ID, TOC_ID, e.Title, e.SessionName])
    counter = counter + 1

text.append(f'\\end{{enumerate}}')

if (len(text) > 0):
    with open(f'./content/content-paper.tex', 'w') as fp:
        fp.write('\n'.join(text))
        
        
with open('./TOC-CHI23.csv', 'w', newline='') as csvfile:
    w = csv.writer(csvfile, delimiter=',',
                            quotechar='"')
    w.writerow(["ID","PaperID","PCS_ID","TOC_ID","Title","Session"])
    for e in tapsTOC:
        w.writerow(e)
        
dfAI = pd.DataFrame(lstAuthorIndex)
dfAI.columns = ["Name", "Submission"]
dfAI = dfAI.groupby("Name").Submission.apply(lambda x: ", ".join(x))
dfAI = dfAI.reset_index()
dfAI["NameRaw"] = dfAI.Name.str.replace('"', '')
dfAI["NameRaw"] = dfAI.NameRaw.apply(lambda x: unidecode(x))
dfAI["NameRaw"] = dfAI.NameRaw.str.replace('[', '', regex=False)
dfAI["NameRaw"] = dfAI.NameRaw.str.lower()
dfAI = dfAI.sort_values("NameRaw")

if (len(dfAI) > 0):
    with open(f'./content/index-paper.tex', 'w') as fp:
        fp.write('\\begin{multicols}{2}\n')
        for i, e in dfAI.iterrows():
            fp.write(f'{e.Name} \dotfill {e.Submission}\\\\\n')
            
        fp.write('\\end{multicols}')

        

Error in 0 papers
WITHDRAWN pn8949
WITHDRAWN pn6732
['i' 'e' 'p' 'm' 'd' 'u' 'a' 'j' 'f' 's' 'r' 't' 'h' 'b' 'w' 'g' 'v' 'c'
 'µ' 'n' 'l' 'o' 'q' '3' 'y' 'k' '1' 'z' '4' 'x']
883
879
879


In [188]:
f'{e.Name} \dotfill {e.Submission}'

'Ángel Alexander Cabrera \\dotfill Paper419'

In [171]:
e

'Name'